Following on from the exploration done in the exploratory_analysis.ipynb notebook, I will work on the actual prediction task for the ionosphere dataset from UCI to be found here: https://archive.ics.uci.edu/ml/datasets/Ionosphere

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rnd

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.cross_validation import KFold, train_test_split
from sklearn.decomposition import PCA

%matplotlib inline
rnd.seed(111)

data = pd.read_csv('ionosphere_processed.csv', index_col=0)
data.drop('1', axis=1, inplace=True)
X = data.iloc[:, :-1].as_matrix()
y = np.array(data.iloc[:, -1])

Since the ionosphere dataset is a classification problem with the target variable taking on either 'good' or 'bad' values, I will start with a logistic regression and then go from there trying to improve on that performance.

In [31]:
logit_classifier = LogisticRegression(solver='liblinear')
logit_classifier.fit(X, y)
yhat = logit_classifier.predict(X)
probabilities = logit_classifier.predict_proba(X)

'yhat' now contains the model's predictions for each observation while 'probabilities' gives each observation's probability to belong to either class. Since this is a two-class classification problem, the probabilities should in all cases sum to 1.

In [32]:
probabilities.sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1

The decision made by the classifier is then simply to predict that an observation belongs to the class with the higher probability. In this case, that makes sense; however, there might also be other contexts in which one would like to classify a given observation as belonging to some class already if it has a probability for that class greater than e.g. a threshold of 30%. The reason might be that the costs for a false negative significantly outweigh the costs of a false positive. 

Let's take a look at how well the classifier did in predicting the target variable:

In [33]:
conf_mat_0 = confusion_matrix(y_true=y, y_pred=yhat)
conf_mat_0

array([[ 97,  29],
       [  5, 220]])

Top-left of the confusion matrix are true positives, top-right are false negatives, bottom-left are false positives, and bottom-right are true negatives. It seems the simple logistic regression classifier is not doing too badly; however, these performance metrics are giving too positive an impression. The reason for that is that I used the entire dataset to train the classifier and subsequently evaluated over the entire dataset, too. This can easily lead to an underestimation of prediction errors and give too 'rosy' a picture. 

In order to avoid this, one could for example split the dataset into two pieces using the first to train the classifier and the second to evaluate it. This can give a better impression of how well the classifier is able to generalize, i.e. predict correct labels for previously-unseen data.

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33,
                                                    random_state=0)

In [35]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(235, 33)
(116, 33)
(235,)
(116,)


The train_test_split() function has helpfully divided the dataset into altogether four pieces: two for the features, and two for the target variable. The samples were randomly selected.

Now I can train the classifier only on the train subset:

In [36]:
logit_classifier = LogisticRegression(solver='liblinear')
logit_classifier.fit(X_train, y_train)
yhat = logit_classifier.predict(X_test)

In [38]:
conf_mat_1 = confusion_matrix(y_true=y_test, y_pred=yhat)
conf_mat_1

array([[34, 15],
       [ 0, 67]])

How does this compare to the confusion matrix from above?

In [42]:
(conf_mat_0[0, 0] + conf_mat_0[1, 1]) / sum(sum(conf_mat_0))

0.90313390313390318

In [44]:
(conf_mat_1[0, 0] + conf_mat_1[1, 1]) / sum(sum(conf_mat_1))

0.87068965517241381

As expected, the fraction of correctly classified observations has gone down, demonstrating that using the entire dataset for training and testing underestimates the classification error for unseen data.

Taking the idea of using subsets of the dataset for training and other subsets for testing one step further leads to what is called k-fold cross-validation (https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation).

In k-fold cross-validation, the dataset is divided into k equally sized parts. The classifier is then trained on k-1 of those and evaluated on the remaining part. This process is repeated k times so that each partition (or fold) is used as a test set once. The prediction error can then be averaged over all k folds in order to better estimate how the classifier would generalize. Let's try this out before moving on from logistic regression.

In [45]:
kf = KFold(n=X.shape[0], n_folds=5, shuffle=True)

In [46]:
list_conf_mat = []
for train_idx, test_idx in kf:
    classifier = LogisticRegression(solver='liblinear')
    classifier.fit(X[train_idx, :], y[train_idx])
    yhat = classifier.predict(X[test_idx, :])
    list_conf_mat.append(confusion_matrix(y_true=y[test_idx],
                                          y_pred=yhat))

In [52]:
for conf_mat in list_conf_mat: 
    print(conf_mat)
    print('')

[[18  8]
 [ 0 45]]

[[14  7]
 [ 2 47]]

[[16  9]
 [ 0 45]]

[[16  9]
 [ 0 45]]

[[16 13]
 [ 2 39]]



In [54]:
np.mean([(cm[0, 0] + cm[1, 1]) / sum(sum(cm)) for cm in list_conf_mat])

0.85746478873239451

On average, the logistic regression correctly classifies about 86% of the observations as evaluated by 5-fold cross-validation. From here on out, I will try to improve on this baseline performance.